In [3]:
from math import exp, sqrt
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from numba import jit

In [4]:
# Define the Franke Fumction
def frankeFunction(x1,x2,sig2):
    noise = np.random.normal(0,sig2,x1.shape)
    term1 = 0.75*np.exp(-(0.25*(9*x1-2)**2) - 0.25*((9*x2-2)**2))
    term2 = 0.75*np.exp(-((9*x1+1)**2)/49.0 - 0.1*(9*x2+1))
    term3 = 0.5*np.exp(-(9*x1-7)**2/4.0 - 0.25*((9*x2-3)**2))
    term4 = -0.2*np.exp(-(9*x1-4)**2 - (9*x2-7)**2)
    return term1 + term2 + term3 + term4 + noise

# Create the data points in mesh grid form
def createDataPoints(n, sig2):
    x1 = np.arange(0, 1, 1/n)
    x2 = np.arange(0, 1, 1/n)
    x1_d, x2_d = np.meshgrid(x1,x2)
    y_d = frankeFunction(x1_d,x2_d,sig2)
    return x1_d, x2_d, y_d.ravel()

@jit
def createDesignMatrix(x1, x2, n=4):
    if len(x1.shape) > 1:
        x1 = np.ravel(x1)
        x2 = np.ravel(x2)

    N = len(x1)
    p = int((n+1)*(n+2)/2)
    X = np.ones((N,p))

    for i in range(1, n+1):
        q = int(i*(i+1)/2)
        for j in range(i+1):
            X[:,q+j] = (x1**(i-j))*(x2**j)
    return X

# Calculate the mean square error (MSE)
def MSE(y_data, y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

# Calculate the coefficient of determination (R2)
def R2(y_data, y_model):
    n = np.size(y_data)
    return 1 - np.sum((y_data-y_model)**2)/np.sum((y_data-(np.sum(y_data)/n))**2)

def Scale(X_train, X_test):
    XX = np.copy(X_test)
    if X_train.shape[1] > 1:
        scaler = StandardScaler()
        scaler.fit(X_train[:,1:])
        X_train[:,1:] = scaler.transform(X_train[:,1:])
        XX[:,1:] = scaler.transform(XX[:,1:])  
    return X_train, XX#, y_train[:, np.newaxis], y_test[:, np.newaxis]


# The stochastic gradient descent method
def SGD(X, y):
    theta_linreg = np.linalg.inv(X.T @ X) @ (X.T @ y)
    
    

In [5]:
etas = [10^-4, 10^-3, 10^-2, 10^-1]
numMiniBatch = []

In [90]:
np.random.seed(26)

sig2 = 0#.01 # noise variance
degree = 5 # polynomial degree
n = 10 # number of data points for each feature
x1, x2, y = createDataPoints(n, sig2)
X_train = createDesignMatrix(x1,x2,degree)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
X_train, X_test = Scale(X_train, X_test)
y_train = y_train[:, np.newaxis]

m = X_train.shape[0] # number of training examples
eta = 0.01
NIterations = 2000000

def olsRegression(X_train,X_test,y_train,y_test, etas, numMiniBatch, CV=True):
    
    beta = np.random.normal(scale=0.1, size=(X_train.shape[1],1))
    
    for i in range(NIterations):
        gradients = 2.0/m*X_train.T @ ((X_train @ beta)-y_train)
        beta -= eta*gradients

    betaAna = np.linalg.pinv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
    y_tilde = (X_train @ beta)
    y_ana = (X_train @ betaAna)
    y_pred = (X_test @ beta)
    if CV == True:
#         print(f"The R2 value for a polynomial of order {degree}, OLS test: {R2(y_test, y_pred)}")
#         print(f"The MSE value for a polynomial of order {degree}, OLS test: {MSE(y_test, y_pred)}")
        print(f"\nThe R2 value for a polynomial of order {degree}, OLS train: {R2(y_train, y_tilde)}")
        print(f"The MSE value for a polynomial of order {degree}, OLS train: {MSE(y_train, y_tilde)}")
        
        print(f"\nThe R2 value for a polynomial of order {degree}, Ana train: {R2(y_train, y_ana)}")
        print(f"The MSE value for a polynomial of order {degree}, Ana train: {MSE(y_train, y_ana)}")
        
    print(beta)
    print()
    print(betaAna)
    return beta, y_pred

_, yy = olsRegression(X_train,X_test,y_train,y_test,1,1)


The R2 value for a polynomial of order 5, OLS train: 0.9729194305382192
The MSE value for a polynomial of order 5, OLS train: 0.0020141301603733897

The R2 value for a polynomial of order 5, Ana train: 0.9787329892861849
The MSE value for a polynomial of order 5, Ana train: 0.001581743979207379
[[ 0.44203531]
 [ 1.00705751]
 [ 0.51564604]
 [-3.88306348]
 [ 0.14270999]
 [-2.00838877]
 [ 1.86866122]
 [ 0.30614536]
 [-0.4468004 ]
 [ 0.58530821]
 [ 2.95268251]
 [-0.4624873 ]
 [ 2.64023932]
 [-1.71623163]
 [ 1.91770716]
 [-2.18391212]
 [-0.1136331 ]
 [-0.41839019]
 [-1.25037788]
 [ 1.39791477]
 [-1.19830279]]

[[ 0.44203531]
 [ 1.24797588]
 [ 0.29523347]
 [-4.49953049]
 [-0.54424774]
 [ 0.10509059]
 [ 1.26031001]
 [ 2.61592339]
 [ 0.24034948]
 [-5.61627528]
 [ 5.34541355]
 [-3.44027975]
 [ 2.29726577]
 [-2.68369269]
 [ 9.27701219]
 [-3.59182501]
 [ 1.11703174]
 [-0.1657703 ]
 [-1.26348447]
 [ 1.89484826]
 [-4.23729917]]


In [160]:
np.random.seed(26)

sig2 = 0#.01 # noise variance
degree = 5 # polynomial degree
n = 10 # number of data points for each feature
x1, x2, y = createDataPoints(n, sig2)
X_train = createDesignMatrix(x1,x2,degree)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
X_train, X_test = Scale(X_train, X_test)
y_train = y_train[:, np.newaxis]

m = X_train.shape[0] # number of training examples
samPerMini = 7
numMiniBatch = np.int(m/samPerMini)
eta = 0.01
    
n_epochs = 1000
t0, t1 = 5, 50
def learning_schedule(t):
    return t0/(t+t1)

def momentumSGD(beta):
    gamma = 0.9
    velocity = 0
    for epoch in range(n_epochs):
        for i in range(numMiniBatch):
#             random_index = np.random.randint(m)
#             xi = X_train[random_index:random_index+1]
#             yi = y_train[random_index:random_index+1]

            xi = X_train[i*samPerMini:(i+1)*samPerMini]
#             print("xi",xi.shape)
            yi = y_train[i*samPerMini:(i+1)*samPerMini]
#             print("yi",yi.shape)
            
            gradients = 2 * xi.T @ ((xi @ beta)-yi)
            velocity = gamma*velocity + (1-gamma)*gradients
    #             eta = learning_schedule(epoch*m+i)
            beta = beta - eta*velocity
    return beta

def olsRegression(X_train,X_test,y_train,y_test, etas, numMiniBatch, CV=True):
    
    beta = np.random.normal(scale=0.1, size=(X_train.shape[1],1))

    beta = momentumSGD(beta)
    
    betaAna = np.linalg.pinv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
    y_tilde = (X_train @ beta)
    y_ana = (X_train @ betaAna)
    y_pred = (X_test @ beta)
    if CV == True:
#         print(f"The R2 value for a polynomial of order {degree}, OLS test: {R2(y_test, y_pred)}")
#         print(f"The MSE value for a polynomial of order {degree}, OLS test: {MSE(y_test, y_pred)}")
        print(f"\nThe R2 value for a polynomial of order {degree}, OLS train: {R2(y_train, y_tilde)}")
        print(f"The MSE value for a polynomial of order {degree}, OLS train: {MSE(y_train, y_tilde)}")
        
        print(f"\nThe R2 value for a polynomial of order {degree}, Ana train: {R2(y_train, y_ana)}")
        print(f"The MSE value for a polynomial of order {degree}, Ana train: {MSE(y_train, y_ana)}")
        
    print(beta)
    print()
    print(betaAna)
    return beta, y_pred

_, yy = olsRegression(X_train,X_test,y_train,y_test,1,1)


The R2 value for a polynomial of order 5, OLS train: 0.9108586277315887
The MSE value for a polynomial of order 5, OLS train: 0.006629931718248028

The R2 value for a polynomial of order 5, Ana train: 0.9787329892861849
The MSE value for a polynomial of order 5, Ana train: 0.001581743979207379
[[ 0.44188995]
 [ 0.13475272]
 [ 0.1960883 ]
 [-0.9056089 ]
 [ 0.20038305]
 [-0.80710424]
 [ 0.11261545]
 [ 0.13802502]
 [-0.16645109]
 [-0.08736562]
 [ 0.52980591]
 [ 0.29866148]
 [ 0.08891222]
 [-0.18540487]
 [ 0.29692551]
 [-0.14036431]
 [-0.34555878]
 [ 0.05127774]
 [-0.0237595 ]
 [ 0.07248473]
 [ 0.19269134]]

[[ 0.44203531]
 [ 1.24797588]
 [ 0.29523347]
 [-4.49953049]
 [-0.54424774]
 [ 0.10509059]
 [ 1.26031001]
 [ 2.61592339]
 [ 0.24034948]
 [-5.61627528]
 [ 5.34541355]
 [-3.44027975]
 [ 2.29726577]
 [-2.68369269]
 [ 9.27701219]
 [-3.59182501]
 [ 1.11703174]
 [-0.1657703 ]
 [-1.26348447]
 [ 1.89484826]
 [-4.23729917]]
